In [13]:
import pandas as pd
data = {'text':[], 'label':[]}
df = pd.read_excel('./sampled.xlsx')
df['DATA'] = df['DATA'].fillna('')
for dt, label in zip(df['DATA'], df['TARGET']):
    try: 
        data['label'].append(int(label))
        data['text'].append(dt)
    except: pass
print(len(data['text']), len(data['label']))
df = pd.DataFrame(data)

499 499


In [14]:
df

,text,label
0,유럽에서는 겨울철만 되면 남여노소 뱅쇼를 즐긴다고 해요~ 와인에 과일을 넣고 푹끓...,1
1,빨간 순두부만 먹다가... 껍질벗긴 들깨가루를 넣어 영양을 더해보니 보양식이 따로 ...,1
2,고기와 양념만 있으면 어디서든 쉽게 만들어 먹을 수 있어요. 캠핑의 꽃은 고기!,0
3,아들이 먹고 싶다는 골뱅이무침에 소면을 곁들여 한끼 식사로 충분,0
4,사과는 식이섬유가 많고 다당류 비율이 높아서 다이어트에도 도움이 되며 특히 사과 ...,0
...,...,...
494,강판에 갈아서 만드는 감자전은 많이 알고 계실텐데요. 감자를 갈지 않고 채를 썰어서...,1
495,http://blog.naver.com/chez_sophie,0
496,파티에 튀김요리 없으면 허전하죠. 언제나 맛있는 튀김요리 모양도 이쁘고 완전 바삭...,0
497,남편이 좋아하는 시원한 재첩국을 미소 된장을 풀어 구수하고 담백하게 끓였는데 정말 ...,1


In [16]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader, Dataset

# 데이터 준비 (예시)
# data = {
#     'text': [
#         '어묵김말이: 맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요',
#         '현미호두죽: 현미호두죽',
#         '참치삼각김밥: 맨날 편의점에서만 사먹는 삼각김밥~ 집에서 간편히 만들 수 있다는 사실~ 참치만 있으면 냉장고 열어서 만들 수 있어요~ 한끼식사로도 좋고 도시락으로도 좋은 참치삼각김밥을 만들어 보아요~'
#         # 추가 데이터 497개...
#     ],
#     'label': [1, 0, 0]  # 추가 데이터 라벨...
# }

# df = pd.DataFrame(data)

# 데이터셋 클래스 정의
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# BERT 토크나이저 및 모델 로드
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-uncased', num_labels=2)

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# 데이터셋 및 데이터로더 준비
train_dataset = TextDataset(X_train.tolist(), y_train.tolist(), tokenizer, max_len=128)
test_dataset = TextDataset(X_test.tolist(), y_test.tolist(), tokenizer, max_len=128)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# 훈련 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=20,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# 모델 훈련
trainer.train()

# 모델 평가
results = trainer.evaluate()
print("Evaluation results:", results)

# 예측 및 평가
preds_output = trainer.predict(test_dataset)
preds = preds_output.predictions.argmax(-1)

print("Accuracy:", accuracy_score(y_test, preds))
print("Classification Report:\n", classification_report(y_test, preds))


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\user\miniconda3\envs\unet\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--bert-base-multilingual-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

{'eval_loss': 1.6156260967254639, 'eval_runtime': 1.2972, 'eval_samples_per_second': 77.088, 'eval_steps_per_second': 5.396, 'epoch': 20.0}
{'train_runtime': 262.8466, 'train_samples_per_second': 30.36, 'train_steps_per_second': 1.902, 'train_loss': 0.2863356233239174, 'epoch': 20.0}


  0%|          | 0/7 [00:00<?, ?it/s]

Evaluation results: {'eval_loss': 1.6156260967254639, 'eval_runtime': 1.2649, 'eval_samples_per_second': 79.06, 'eval_steps_per_second': 5.534, 'epoch': 20.0}


  0%|          | 0/7 [00:00<?, ?it/s]

Accuracy: 0.72
Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.74      0.75        58
           1       0.66      0.69      0.67        42

    accuracy                           0.72       100
   macro avg       0.71      0.72      0.71       100
weighted avg       0.72      0.72      0.72       100



In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

text = "새콤하고 달콤하니 맛있는 오이무침이에요"

# 토크나이징 및 텐서 변환
inputs = tokenizer(text, return_tensors="pt")
inputs = inputs.to(device)
outputs = model(**inputs)
print(outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[-3.3857,  3.7761]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
